# AMIA 2016 Annual Symposium Workshop (WG13)

### Mining Large-scale Cancer Genomics Data Using Cloud-based Bioinformatics Approaches (RNAseq)

Riyue Bao, Ph.D. 
Center for Research Informatics,
The University of Chicago.
8:45 AM - 10:00 AM, November 13, 2016

***

## Table of Contents: <a class="anchor" id="top"></a>
* [Objective](#01)
* [Dataset](#02)
* [1. RNAseq technology, clinical application and data analysis](#03)
	* [1.1 What is RNAseq](#03.1)
	* [1.2 Clinical applications](#03.2)
* [2. How to perform RNAseq analysis](#04)
	* [Workflow](#04.1)
	* [2.1 - 2.2 Quality assessment and preprocessing of raw sequencing reads](#04.2)
	* [2.3 - 2.4 Map reads to reference genome and quantification of transcript abundance](#04.3)
	* [2.5 - 2.7 Normalize gene expression and identify DEGs: DESeq2](#04.4)
	* [2.8 Identify enriched GO and pathways: clusterProfiler](#04.5)
* [3. Practice how to associate gene expression with clincial data](#05)
	* [Workflow](#05.1)
	* [3.1 - 3.2. Import data, normalize expression and cluster patients based on expression profile](#05.2)
	* [3.3 Survival analysis](#05.3)
* [Summary](#06)
* [Run pipeline (after the workshop)](#07)

***

## Objective<a class="anchor" id="01"></a> <small>[[top](#top)]</small>

* **Introduction to RNAseq technology and clinical application** (10 minutes)
    * Learn the background and when to use RNAseq for your projects
* **How to perform RNAseq analysis** (50 minutes) *hands-on*
    * Learn the good-practice analysis protocol (**8 steps**)
        * From raw sequencing data to quantitative measures of transcripts (automated pipeline)
        * Identify differentially expressed genes (DEGs) between conditions and affected pathways (R & Bioconductor)
        * Generate high-quality figures for publication
* **Associate gene expression data with clincial outcome** (15 minutes) *hands-on*
    * Discover tumor subtypes and detect survival difference (**3 steps**) (R & Bioconductor)
    * The Cancer Genome Atlas (TCGA), hosted at NCI Genomics Data Commons (GDC)

<img src='notebook_ext/ipynb_data/assets/Figure24.png', title = 'RNAseq objective', width = 600, height = 600>

***

## Dataset<a class="anchor" id="02"></a> <small>[[top](#top)]</small>
* **Data for RNAseq analysis**
    * Two groups (*PRDM11* KO vs WT, U2932 cells), 6 samples
    * Aim to identify DEGs / pathways altered between KO and WT groups
    * Fog. et al. 2015. [Loss of PRDM11 promotes MYC-driven lymphomagenesis](http://www.bloodjournal.org/content/125/8/1272.long?sso-checked=true). Blood 125(8):1272-81   

| Sample | Group | Sequencing File | Sequencing Data |
|------|------|------|------|------|   
| KO01 | KO | KO01.fastq.gz | 74,126,025 reads |   
| KO02 | KO | KO02.fastq.gz | 64,695,948 reads |   
| KO03 | KO | KO03.fastq.gz | 52,972,573 reads |   
| WT01 | WT | WT01.fastq.gz | 55,005,729 reads |   
| WT01 | WT | WT02.fastq.gz | 61,079,377 reads |   
| WT01 | WT | WT03.fastq.gz | 66,517,156 reads |  

* **Data for clinical association**
    * TCGA ovarian cancer, 379 primary tumors **[Genomics Data Commons](https://gdc-portal.nci.nih.gov)**
    * Aim to use gene expression to group patients into subtypes and detect survival differences
    * The Cancer Genome Atlas Research Network. 2011. [Integrated genomic analyses of ovarian carcinoma](http://www.nature.com/nature/journal/v474/n7353/full/nature10166.html). Nature 474, 609–615
<img src='notebook_ext/ipynb_data/assets/Figure23.2.png', title = 'GDC', width = 600, height = 600>

***

## 1. RNAseq technology, clinical application and data analysis<a class="anchor" id="03"></a> <small>[[top](#top)]</small>

### 1.1 What is RNAseq<a class="anchor" id="03.1"></a> <small>[[top](#top)]</small>

* **High-throughput sequencing of RNA: Profile, identify or assemble transcripts**
    * Detect gene / isoform expression changes between conditions
    * Identify novel splice sites / exons, mutations, fusion genes, etc.
    * Broad detection range (lowly to highly expressed genes), high sensitivity, low requirement of RNA amount
    * Available for all species (reference genome is optional): reference genome-guided alignment or *de novo* assembly
    * Transcriptome-wide approach for quantitative measurements and gene discovery without prior knowledge
<img src='notebook_ext/ipynb_data/assets/Figure14.png', title = 'A typical RNAseq experiment', width = 800, height = 400>
* **RNAseq cannot answer**
    * Transcription factor binding sites / histone modification on the genome (ChIPseq)
    * DNA/histone methylation changes (Methylated DNA immunoprecipitation sequencing (MeDIPseq), Bisulfite sequencing)
    * Genomic DNA changes (Whole genome/exome sequencing (WGS/WXS), targeted panel DNAseq)
    * Chromatin interactions (5C, 7C)
    * Protein / peptide identification and quantification (isobaric peptide labelling approach (iTRAQ))
* **Experimental Design** ([considerations and limitations](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq/tree/master/notebook_ext/ipynb_data/assets/Figure13.png))
    * Include biological replicates for increased discovery power
<img src='notebook_ext/ipynb_data/assets/Figure15.png', title = 'Biological replicates', width = 400, height = 400>

***
### 1.2 Clinical applications<a class="anchor" id="03.2"></a> <small>[[top](#top)]</small> 
* Discover gene signature that predicts patient's clinical outcome
* Detect gene fusions in tumor samples as causal genomic aberrations for disease
* Integrate with other types of genomic data (e.g. ChIPseq, miRNAseq) to identify tumor types
* Challenges and limitations
    * Relatively poor RNA quality for tumor FFPE samples
    * Contamination from adjacent normal tissue
    * Still more expensive than targeted-panel sequencing such as NanoString
    * 40 million mapped reads are usually sufficient for gene profiling, but > 80 millions are required to detect bottom 1% lowly expressed genes
* **TCGA datasets** (35 cancer types, > 20,000 tumor/normal samples, > 9,000 patients)
<img src='notebook_ext/ipynb_data/assets/Figure17.png', title = 'TCGA OV Nature paper', width = 600, height = 90>

***

## 2. How to perform RNAseq analysis<a class="anchor" id="04"></a> <small>[[top](#top)]</small>

<img src='notebook_ext/ipynb_data/assets/Figure20.png', title = 'RNAseq analysis', width = 800, height = 400>

### Workflow 

The best-practice analysis protocol takes 8 major steps. For a more detailed description, refer to the extended version of notebooks (`notebook_ext/02.Run_RNAseq.tutorial.ipynb`).

* Steps 1-4: From raw sequencing to transcript quantification is **automated through [BigDataScript](https://pcingola.github.io/BigDataScript/) (BDS) pipeline**.
* Steps 5-8: DEG and pathway analysis will be practiced **interactively to better interpret the results**.

<img src='notebook_ext/ipynb_data/assets/Figure25.png', title = 'RNAseq workflow', width = 900, height = 900>

Due to time limit, we will not run the BDS pipeline in this workshop. Since it is automated, participants are encouraged to practice it post-workshop following the instructions at [Github](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq). The launching of the pipeline only requires submission of one script `Build_RNAseq.sh`, which was already prepared for you. The pipeline was designed to **take care of all the dependencies between tasks/jobs with robust checkpoints and high reproducibility**. 

### Steps 1-4

***

### 2.3.1 - 2.3.2 Quality assessment and preprocessing of raw sequencing reads<a class="anchor" id="04.2"></a> <small>[[top](#top)]</small>

* **Raw sequencing reads** are stored in FastQ format (e.g. `KO01.fastq.gz`), where each read is presented by 4 lines
<img src='notebook_ext/ipynb_data/assets/Figure10.png', title = 'Sequencing reads in FastQ format', width = 600, height = 90>
* QC produces reports that help you evaluate if a sequencing run is successful and if reads are of high quality ([example](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq/tree/master/notebook_ext/ipynb_data/assets/multiqc_report.html))
* *Optional* Preprocess reads to improve mapping rate and accuracy (in the next step, 1.3.3)
    * Trim low-quality bases, clip adapters, etc.
    * Avoid overtrimming in RNAseq! ([Williams et al. 2016](http://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-0956-2))

***
### 2.3.3 - 2.3.4 Map reads to reference genome and quantification of transcript abundance<a class="anchor" id="04.3"></a> <small>[[top](#top)]</small>
* Read mapping identifies the location in the genome where a sequencing read comes from
* Accurate mapping result is the key for quantification and DEG identification
* **splice-aware aligner** (e.g. [STAR](https://github.com/alexdobin/STAR))
<img src='notebook_ext/ipynb_data/assets/Figure12.png', title = 'RNAseq mapping result', width = 600, height = 400>
* **Different aligners may generate very different results** ([Engström et al. 2013](http://www.nature.com/nmeth/journal/v10/n12/full/nmeth.2722.html))

***
### RNAseq metrics
* Collect metrics to evaluate RNA sample quality and identify potential problems ([example](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq/tree/master/notebook_ext/ipynb_data/assets/multiqc_report.html))
    * **Is there high-level genomic DNA contamination?**
    * **Was ribosome RNA sucessfully depleted during library prep?**
    * Is the RNA highly degraded?
    * How do reads distribute on the genome? (exons, introns, intergenic, etc.)
    * Is the strandness of read alignment consistent with library type? (non-stranded or forward/reverse strand-specific)
    * Does the target gene knocked down in KO samples have reduced expression as expected?
<blockquote>
Q1: Which sample (S1-4) has the most severe contamination from genomic DNA?   
Hint: higher percentage of intergenic reads indicates more severe DNA contamination in RNA samples
</blockquote>
<blockquote>
Q2: Which sample (S1-4) has the least efficient depletion of ribosome RNAs (rRNAs)?   
Hint: rRNAs account for > 80% of the whole transcriptome. If not removed, the majority of the sequencing reads will be derived from rRNA and not the coding or lincRNAs.
</blockquote>
<img src='notebook_ext/ipynb_data/assets/Figure5.png', title = 'Figure5', width = 600, height = 600>
*For answers to other questions, refer to the extended version of notebooks* (`notebook_ext/02.Run_RNAseq.tutorial.ipynb`)

### Steps 5-8

* From the pipeline run, we generated read alignment and counts for every annotated gene on the genome
* The next step is to utilize the read counts data to detect DEGs
* For example, if we visualize *FOS* gene across 6 samples in genome browser (e.g. [IGV](http://software.broadinstitute.org/software/igv/))

<img src='notebook_ext/ipynb_data/assets/Figure16.png', title = 'KO vs WT read coverage FOS', width = 600, height = 600>

## Hands-on practice START

***
### 2.3.5 - 2.3.7 Normalize gene expression and identify DEGs: DESeq2<a class="anchor" id="04.4"></a> <small>[[top](#top)]</small>

Involves five sub-steps in [R](https://www.r-project.org/about.html) (i to v).

**To systematically identify significant DEGs across the transcriptome, we will use [DESeq2](http://bioconductor.org/packages/release/bioc/html/DESeq2.html)**
* Different tools (`edgeR`, `limmavoom`) may have very different performance! [Seyednasrollah F et al. 2015](https://dx.doi.org/10.1093/bib/bbt086)
* We will also practice how to generate publication-quality figures such as PCA, heatmap, clustering, and pathways from our data

**To facialiate workshop demo, small R script snippets were prepared to simplify steps described in hands-on**
* Those scripts can be found in directory `notebook_ext/ipynb_data/Rscripts`, e.g. `DESeq2.plot_PCA.R`.

**Good coding practice**
* One statement each line for clarity. It is possible to concatenate statements with `;` but not recommended
* Break long statements into multiple lines with less than 80 column per line for better readability!

***
### i. Prepare working environment and set up parameters
* Set up global variables at the beginning, avoid hard-coding of file names, directories, etc.
* Can easily apply to other projects with similar experimental design

In [ ]:
##-- Clean the environment: always run this before starting new analysis!
rm(list=ls())

##-- Load libraries required in this analysis
pkg.list = c('ggplot2', 'RColorBrewer', 'reshape',
             'DESeq2', 'vsn', 'genefilter', 'pheatmap',
             'clusterProfiler', 'pathview', 'IRdisplay')
for(pkg in pkg.list) {
    suppressMessages(require(pkg, character.only = TRUE))
}

##-- Parameters
cancer = 'DLBC' 
fdr = 0.05
fc = 1.5
gene.type = 'coding'
caller = 'deseq2'
group1 = 'KO'
group2 = 'WT'
colors = c('#CC0000', '#00CC00')

##-- Set up working directory
work.dir = '.'
setwd(work.dir)

##-- Input/Output directories
in.dir = 'notebook_ext/ipynb_data/input'
out.dir = 'notebook_ext/ipynb_data/output'

##-- Input/Output files
expr.file = paste0(cancer, '.raw_counts.coding.tsv')
sample.file = paste0(cancer, '.sample_group.tsv')

##-- Print analysis info
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.print_info.R')
source(script)

***
### ii. Import data
* Read counts on coding genes 
* Sample groups

In [ ]:
##-- Read data files
data.expr = read.delim(file.path(in.dir, expr.file), header=T, stringsAsFactors=F)
data.sample = read.delim(file.path(in.dir, sample.file), header=T, stringsAsFactors=F)

data.expr[1:3,]
data.sample

***
### iii. Principle Component Analysis (PCA) and biological replicate correlation
* Both are good assessment to identify mislabeled samples and outliers
* All genes are used as input!
* PCA - do samples group as expected? 
* Correlation - do biological replicates correlate with each other? 

In [ ]:
##-- Format data matrix
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.format_data.R')
source(script)

##-- DESeq2: build DESeqDataSet object, prepare design matrix
dds = DESeqDataSetFromMatrix(countData = as.matrix(data.expr.proc),
                             colData = data.sample, 
                             design = ~ Group)
# print(dds)

##-- DESeq2: normalize matrix for clustering/heatmap generation
vsd = varianceStabilizingTransformation(dds, blind=FALSE)

##-- DESeq2: sample correlation heatmap
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.plot_SMcor_heatmap.R')
source(script)

##-- DESeq2: Principal component analysis (PCA) plot
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.plot_PCA.R')
source(script)

***
### iv. Normalize and detect DEGs
* Normalize read counts and fit a linear model
* Lowly expressed genes are automatically filtered out (default as mean read counts < 10)
* Estimate the **KO vs WT** fold-change and p-values for each gene 

In [ ]:
##-- DESeq2: set up variables
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.set_variable.R')
source(script)

##-- DESeq2: fit the model and identify DEGs
dds = DESeq(dds, test="Wald", betaPrior=T)
res = results(dds, 
              contrast=c("Group",group1,group2), 
              pAdjustMethod ="fdr", 
              alpha=fdr)

##-- DESeq2: peek into DEG data object
summary(res)
res = as.data.frame(res)

##-- DESeq2: add anti-log2 fold change and gene symbol to output
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.add_fc_symbol.R')
source(script)

***
### v. Filter for significant DEGs and plot heatmap
* Filter for significant DEGs with at least 1.5 fold-change and FDR-adjusted p-value < 0.05
* Generate expression heatmap on significant DEGs (different from PCA and correlation heatmap described in iii)

In [ ]:
##-- DESeq2: filter significant DEGs by fc 1.5 and adjusted p < 0.05
res[1:3,]
script = file.path('notebook_ext','ipynb_data','Rscripts','DESeq2.flt_sigDEGs.R')
source(script)

##-- DESeq2: plot sig DEG heatmap
print(paste0('Heatmap = ', nrow(data.plot), ' genes on the row, ', 
             ncol(data.plot), ' samples on the column'))
pheatmap(data.plot,
         scale = 'row',
         cluster_rows = TRUE, cluster_cols = TRUE,
         show_rownames = FALSE, show_colnames = TRUE,
         clustering_distance_rows = "euclidean",
         clustering_distance_cols = "euclidean", 
         clustering_method = "complete",
        )

***
### 2.3.8 Identify enriched GO and pathways: clusterProfiler<a class="anchor" id="04.5"></a> <small>[[top](#top)]</small>
* Gene Ontology bological processes and KEGG pathways
* Enrichment in significant DEGs

In [ ]:
##-- clusterProfiler: NOTE those commands were tested for clusterProfiler 
##-- version 3.0.5 or above. Older versions may not work as they have 
##-- options revised/removed in the new versions.

##-- clusterProfiler: prepare input for GO/KEGG enrichment analysis
script = file.path('notebook_ext','ipynb_data','Rscripts','clusterProfilter.prepare_input.R')
source(script)

##-- clusterProfiler: GO over-representation test
##-- takes ~2 minutes .. skip in workshop!
# ego = enrichGO( gene          = genes.sig.anno$ENTREZID,
#                 universe      = genes.all.anno$ENTREZID,
#                 OrgDb         = 'org.Hs.eg.db',
#                 ont           = "BP",
#                 pAdjustMethod = "fdr",
#                 pvalueCutoff  = 0.05,
#                 readable      = TRUE)
##-- instead, load already generated output
load(paste0(out.prefix, '.flt.fdr', fdr,'_fc',fc, '.enrichGO.RData'))
summary(ego)[1:3,1:7]

##-- clusterProfiler: KEGG over-representation test
##-- takes ~2 minutes .. skip in workshop!
# kk =  enrichKEGG(gene          = genes.sig.anno$ENTREZID,
#                  universe      = genes.all.anno$ENTREZID,
#                  organism      = "human",
#                  pAdjustMethod = "fdr",
#                  pvalueCutoff  = 0.05)
##-- instead, load already generated output
load(paste0(out.prefix, '.flt.fdr', fdr,'_fc',fc,'.enrichKEGG.RData'))
summary(kk)[1:3,]

##-- clusterProfiler: visualize pathway with all genes
gene.list = genes.all.anno$foldChange
names(gene.list) = genes.all.anno$ENTREZID
pathway = 'hsa04010'
plot.title = paste0(pathway, '.gene_all.pathview.png')
p4 = pathview(gene.data  = gene.list,
              pathway.id = pathway,
              species    = "human",
              limit      = list(gene=4, cpd=1),
              kegg.dir   = paste0(out.dir,'/',caller),
              out.suffix = paste0('gene_all.pathview'))

##-- clusterProfiler: visualization of GO and KEGG enrichment results
barplot(ego, showCategory=20)
dotplot(ego, showCategory=20)

In [ ]:
##-- KEGG pathwayview written into local files. Display the plot in R
display_png(file='notebook_ext/ipynb_data/assets/hsa04010.gene_all.pathview.png')

***

## 3. Practice how to associate gene expression with clincial data<a class="anchor" id="05"></a> <small>[[top](#top)]</small>  

* TCGA hosted on GDC (https://gdc-portal.nci.nih.gov)
<img src='notebook_ext/ipynb_data/assets/Figure19.png', title = 'GDC website', width = 800, height = 200>
* **Our data**
    * ~500 Ovarian Cancer patients (expression and clinical)
<img src='notebook_ext/ipynb_data/assets/Figure18.png', title = 'TCGA OV Nature paper', width = 800, height = 200>

### Workflow 

The analysis takes 3 major steps. For a more detailed description, refer to the extended version of notebooks (notebook_ext/04.Run_RNAseq.tutorial.ipynb).

<img src='notebook_ext/ipynb_data/assets/Figure26.png', title = 'TCGA & Clinical workflow', width = 900, height = 900>

***
### 3.1 - 3.2. Import data, normalize expression and cluster patients based on expression profile<a class="anchor" id="05.2"></a> <small>[[top](#top)]</small>

Involves two sub-steps in [R](https://www.r-project.org/about.html) (i to ii).
### i. Import data and select top 150 most variable genes for clustering
* Why selecting most variable genes and not all genes?
    * Without patient groups known, using all genes for sample clustering is computational intensive and may mask the true differences between potential sample groups.
    * The most variable genes are the the biggest contributors to sample variation. 
    * They likely represent genes differentially expressed between potential sample groups from the cohort.

In [ ]:
##-- Clean up environment
rm(list=ls())
set.seed(13)

##-- Load libraries required in this analysis
pkg.list = c('ggplot2', 'RColorBrewer', 'reshape',
             'gplots', 'survival', 'NMF', 'IRdisplay')
for(pkg in pkg.list) {
    suppressMessages(require(pkg, character.only = TRUE))
}

##-- Parameters
cancer = 'OV' 
gene.top.count = 150

##-- Import data
script = file.path('notebook_ext','ipynb_data','Rscripts','TCGA_OV.import_data.R')
source(script)

##-- Preprocess data
script = file.path('notebook_ext','ipynb_data','Rscripts','TCGA_OV.preprocess_data.R')
source(script)

***
### ii. Cluster and visualize patient groups based on gene expression profile

* Visualize NMF clustering results (samples & genes)
* The plotting takes ~2 minute and lots of memory
* In this workshop, we will skip the code and directly visualize the already generated plots
* Note: **This code block still needs to be run to get variables ready for the next step!

In [ ]:
##-- run NMF to cluster samples & genes 
##-- (use 4 core 'p4', and print more info 'v')
##-- takes 5 minutes to run... skip in workshop
print('NMF clustering start...')
# expr.sub.nmf = nmf(expr.sub, 
#                    rank = 3, 
#                    method = 'brunet', 
#                    seed = 'random', 
#                    nrun = 100, 
#                    .opt = 'vp4') 
##-- workshop only: load already generated result
load(paste0(out.dir,'/',expr.file,'.nmf.RData'))
expr.sub.nmf=expr.sub.nmf
print('Done!')

##-- Retrieve sample & gene clusters
##-- add to clinical table
script = file.path('notebook_ext','ipynb_data','Rscripts','TCGA_OV.retrieve_cluster.R')
source(script)
head(clinical)

##-- Plot sample correlation and gene expression heatmaps (skip in workshop)
script = file.path('notebook_ext','ipynb_data','Rscripts','TCGA_OV.plot_heatmap.R')
source(script)

##-- directly view pre-generated heatmaps (workshop only)
display_png(file='notebook_ext/ipynb_data/assets/Figure22.png')

***
### 3.3 Survival analysis<a class="anchor" id="05.3"></a> <small>[[top](#top)]</small>

Involves two sub-steps in [R](https://www.r-project.org/about.html) (i to ii).

* Identify overall survival (OS) difference between the three tumor subtypes / patient groups
* Kaplan–Meier (KM) plot, log-rank test, and cox proportional hazards model (univariate and multivariate)

### i. KM plot and log-rank test

In [ ]:
separator = '========================================'

##-- add right censoring
table(clinical$vital.status)
clinical$censor = NA
clinical$censor[which(clinical$vital.status == 'LIVING')] = 0
clinical$censor[which(clinical$vital.status == 'DECEASED')] = 1
clinical$censor = as.numeric(clinical$censor)

##-- KM plot 
surv = Surv(clinical$overall.survival.month, clinical$censor)
surv.fit = survfit(surv ~ clinical$cluster)
script = file.path('notebook_ext','ipynb_data','Rscripts','TCGA_OV.plot_KM.R')
source(script)

separator

##-- log-rank test
print(paste0('Running log-rank test for survival'))
surv = Surv(clinical$overall.survival.month, clinical$censor)
surv.diff = survdiff(surv ~ clinical$cluster)
surv.diff

separator

***
### ii. Cox multi- and univariant model

In [ ]:
##-- cox pp haz model (comparing cluster 2 and 3 as an example!)
print(paste0('Running Cox proportional hazards model for survival'))
clinical.sub = clinical[clinical$cluster %in% c(2,3),]
surv = Surv(clinical.sub$overall.survival.month, clinical.sub$censor)

##-- full cox model
print(paste0('(a) Full multivariate cox model for survival'))
coxph(surv~(clinical.sub$cluster + 
              clinical.sub$age.at.diagnosis.year + 
              clinical.sub$tumor.grade))

separator

##-- reduced cox model
print(paste0('(b) Reduced multivariate cox model for survival'))
coxph(surv~(clinical.sub$cluster + clinical.sub$age.at.diagnosis.year))

separator

##-- univariate cox model 
print(paste0('(c) Univariate cox model for survival'))
coxph(surv~(clinical.sub$cluster))

***
## Hands-on practice END

In [ ]:
print('Program finished!')
sessionInfo()

***

## Summary<a class="anchor" id="06"></a> <small>[[top](#top)]</small>

In this workshop, we demonstrated how to run RNAseq analysis to identify DEGs & pathways, and how to use expression data to identify patient groups with different clincial outcomes. We introduced the commonly used bioinformatics tools and good-practice approaches. All the analysis were performed on a pre-built machine in AWS EC2 cloud. 

All the class materials will stay open on [GitHub](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq) if you want to practice or use the pipelines and scripts for your own research projects.

Questions? Post on Github or contact Riyue (Sunny) at `rbao AT bsd dot uchicago dot edu`.

<img src='notebook_ext/ipynb_data/assets/Figure21.png', title = 'RNAseq summary', width = 600, height = 600>

***

## Run pipeline (after the workshop)<a class="anchor" id="07"></a> <small>[[top](#top)]</small>

### 1. Open terminal from Jupyter Notebook

Go to your Jupyter Notebook homepage. Click [New] button at top right corner of the homepage. In the dropdown menu, click [Terminal]. 

### 2. Get familiar with the file structure

All workshop materials are openly accessible via [GitHub](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq). Documentation and tutorials of the pipelines can be found in [README](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq) and [wiki](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq/wiki).

* **Run_RNAseq.tutorial.ipynb**: The main lecture / hands-on notebook (covered in workshop)
* **pipeline** directory: the automated, scalable and modularized pipeline 
    * From raw sequencing reads to quantified transcript abundance through one click
    * [`BigDataScript`](https://pcingola.github.io/BigDataScript/) & `Perl`
* **notebook_ext** directory: Extended notebooks 01-04 (not covered in workshop)
    * lecture notes
    * R codes and comments 

`##-- commands`   
`pwd`   
`cd ~/dev/rnaseq/CRI-Workshop-Nov2016-RNAseq`   
`ll pipeline`   
`ll notebook_ext`   

### 2. Launch pipeline (takes ~ 5 minutes)

`##-- commands `   
`pwd`   
`cd ~/dev/rnaseq/CRI-Workshop-Nov2016-RNAseq/pipeline`   
`ll`   
`cd test`   
`./Build_RNAseq.DLBC.sh &`   
`##-- START Thu Oct 27 15:57:38 UTC 2016  Running ../Build_RNAseq.pl`   
`##-- START Thu Oct 27 15:57:38 UTC 2016  Running Submit_RNAseq.DLBC.sh`   
`##-- running ... 3 ~ 4 minutes`   
`##-- END Thu Oct 27 16:01:25 UTC 2016`   
`ll`   

***

## Next Session: ChIPseq analysis and Integration of RNAseq & ChIPseq (Kyle Hernandez PhD)